<a href="https://colab.research.google.com/github/SwathiR1999/building-RNN/blob/main/building_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/SwathiR1999/building-RNN.git
%cd building-RNN
!ls
! git pull

Cloning into 'building-RNN'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (4/4), 23.78 KiB | 624.00 KiB/s, done.
/content/building-RNN
Google_Stock_Price_Test.csv  Google_Stock_Price_Train.csv
Already up to date.


# Importing libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importing datasets

In [3]:
df_train=pd.read_csv('Google_Stock_Price_Train.csv')
train_set=df_train.iloc[:,1:2].values

# Feature Scaling

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))
train_set_scaled = scaler.fit_transform(train_set)

# Creating a data structure with 60 timesteps and 1 output

In [5]:
x_train = []
y_train = []
for i in range(60,1258):
  x_train.append(train_set_scaled[i-60:i,0])
  y_train.append(train_set_scaled[i,0])
x_train,y_train=np.array(x_train), np.array(y_train)

# Reshaping

In [6]:
x_train=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))